In [1]:
import pygame, sys, random
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import math
import matplotlib.pyplot as plt
import pylab
from fractions import Fraction
import time

num_episodes = 1

obstacleRadius = 10
agentRadius = 10

# get size of state and action from environment

boundaryPos = [100, 100]
boundaryLength = [70,70]
boundaryRadius = 40
dispSize = [1280, 960]
initPosAgentStandard = [dispSize[0] - 100, 100]#dispSize[1]]
initPosAgent = initPosAgentStandard#[boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]
goalPos = [100, 900]
goalAngle = 0#random.randrange(0, 360) * math.pi / 180

obstacleRandomRange = 1000

moveObstacles = True
action_size = 9
obsNumber = 200
state_size = 2
# state_size = obsNumber + 1

Using TensorFlow backend.


In [2]:
# A2C(Advantage Actor-Critic) agent
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.load_model1 = True
        self.load_model2 = False
        
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.99
        self.actor_lr = 0.00002
        self.critic_lr = 0.00005

        # create model for policy network
        self.actor = self.build_actor()
        self.critic = self.build_critic()
        self.actorMacro = self.build_actorMacro()
        self.criticMacro = self.build_criticMacro()

        if self.load_model1:
            self.actor.load_weights("./Practice004_DataSave/Actor_Rev.h5")
            self.critic.load_weights("./Practice004_DataSave/Critic_Rev.h5")
            
        if self.load_model2:
            self.actorMacro.load_weights("./Practice004_DataSave/Actor_Macro.h5")
            self.actorMacro.load_weights("./Practice004_DataSave/Critic_Macro.h5")
#             print self.actor.weights
#             print self.critic.weights
    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        actor.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_normal'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy', optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        critic.add(Dense(self.value_size, activation='linear', kernel_initializer='glorot_normal'))
        critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic

    def build_actorMacro(self):
        actorMacro = Sequential()
        actorMacro.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        actorMacro.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_normal'))
        actorMacro.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actorMacro.compile(loss='categorical_crossentropy', optimizer=Adam(lr=self.actor_lr))
        return actorMacro

    # critic: state is input and value of state is output of model
    def build_criticMacro(self):
        criticMacro = Sequential()
        criticMacro.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        criticMacro.add(Dense(self.value_size, activation='linear', kernel_initializer='glorot_normal'))
        criticMacro.summary()
        criticMacro.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return criticMacro
    
    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
#         print policy
        return policy
#         return np.random.choice(self.action_size, 1, p=policy)[0]

    def get_actionMacro(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
#         print policy
        return policy
#         return np.random.choice(self.action_size, 1, p=policy)[0]

    # update policy network every episode
    def train_model(self, state, action, reward, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages = np.zeros((1, self.action_size))

        value = self.critic.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]

        if done:
            advantages[0][action] = reward - value
            target[0][0] = reward
        else:
            advantages[0][action] = reward + self.discount_factor * (next_value) - value
            target[0][0] = reward + self.discount_factor * next_value

        self.actor.fit(state, advantages, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)

In [3]:
def stateGenerator(obsPosition, agtPosition, idx):
    returnSum = []
    if idx != -1:
        returnSum = returnSum + [agtPosition[0] - obsPosition[idx][0], agtPosition[1] - obsPosition[idx][1]]
    else:
        returnSum = returnSum + [agtPosition[0] - obsPosition[0], agtPosition[1] - obsPosition[1]]
    returnSum = np.reshape(returnSum, [1, state_size])
    return returnSum

In [4]:
def takeAction(action):
    xAction = 0
    yAction = 0
    if action == 0:
        xAction = 1
    elif action == 1:
        xAction = 1
        yAction = 1
    elif action == 2:
        xAction = 1
        yAction = -1            
    elif action == 3:
        xAction = -1
        yAction = 1
    elif action == 4:
        xAction = -1
    elif action == 5:
        xAction = -1
        yAction = -1
    elif action == 6:
        yAction = -1
    elif action == 7:
        yAction = 1
    elif action == 8:
        xAction = 0
        yAction = 0
        
    return [xAction, yAction]

In [5]:
def rangeFinder(allObsPos, rangeCenter):
    countObs = 0
    rangeObstacle = [[0,0] for _ in range(obsNumber)]
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
        if math.sqrt((rangeCenter[0] - allObsPos[i][0])**2 + (rangeCenter[1] - allObsPos[i][1])**2) < boundaryRadius:
            rangeObstacle[countObs] = allObsPos[i]
            countObs += 1
    index = np.argmin(allObsAgtDistance)
    return [countObs, rangeObstacle, index]

In [6]:
def goalFinder(agtPos):
    if goalPos[0] == agtPos[0]:
        if goalPos[1] > agtPos[1]:
            goalAngle = 90 * math.pi / 180
        else:
            goalAngle = -90 * math.pi / 180
    else:
        goalAngle = math.atan(1.0*(goalPos[1]-agtPos[1])/(goalPos[0]-agtPos[0]))
    if goalPos[0] < agtPos[0]:
        goalAngle += math.pi
        
    tmpGoal = [0,0]
    tmpGoal[0] = int(math.floor(agtPos[0] + boundaryRadius * math.cos(goalAngle)))
    tmpGoal[1] = int(math.floor(agtPos[1] + boundaryRadius * math.sin(goalAngle)))
    return tmpGoal

In [7]:
def nearestAction(actionIdx):
    nearAction = []
    if actionIdx == 0:
        nearAction = [1, 2]
    elif actionIdx == 1:
        nearAction = [0, 7]
    elif actionIdx == 2:
        nearAction = [0, 6]
    elif actionIdx == 3:
        nearAction = [4, 7]
    elif actionIdx == 4:
        nearAction = [3, 5]
    elif actionIdx == 5:
        nearAction = [4, 6]
    elif actionIdx == 6:
        nearAction = [5, 2]
    elif actionIdx == 7:
        nearAction = [1, 3]
    else:
        nearAction = [8, 8]
    return nearAction

In [8]:
pygame.init()
screen = pygame.display.set_mode(dispSize)
screen.fill([200, 200, 200])

# make A2C agent
agent = A2CAgent(state_size, action_size)
# agentMacro = A2CAgent(state_size, action_size)
rList, episodes = [], []

# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
for i in range(0,obsNumber):
#     obsRadius = random.randrange(agentRadius + obstacleRadius + 10, obstacleRandomRange)
#     obsAngle = random.randrange(90,180) * math.pi / 180
#     obstaclePos[i][0] = int(initPosAgent[0] + obsRadius * math.cos(obsAngle)) #boundaryPos[0] + random.randrange(1, boundaryLength[0])
#     obstaclePos[i][1] = int(initPosAgent[1] + obsRadius * math.sin(obsAngle)) #boundaryPos[1] + random.randrange(1, boundaryLength[1])
#    obstaclePos[i][0] = int(initPosAgent[0] - (obstacleRadius + agentRadius + random.randrange(0, dispSize[0])))
#    obstaclePos[i][1] = int(initPosAgent[1] + (obstacleRadius + agentRadius + random.randrange(0, dispSize[1])))
#     obstaclePos[i][0] = int(150 + i % 14 * 80)
#     obstaclePos[i][1] = int(i / 14 * 80)
    obstaclePos[i][0] = int(150 + i % 7 * 150)
    obstaclePos[i][1] = int(150 + i / 7 * 150)
for e in range(num_episodes):
    # Initialize
    start = time.time()
    frame = 0
    done = False
    score = 0
    x = initPosAgent[0]
    y = initPosAgent[1]
    print("Episode ", e, "Starts!")
    macroPolicy = []
    macroState = []
    [rangeObsNumber, rangeObsPos, minIndex] = rangeFinder(obstaclePos, initPosAgent)
    macroState = stateGenerator([obstaclePos[minIndex][0], obstaclePos[minIndex][1]], [x, y], -1)
    while not done:
#         print rangeObsNumber
#         macroPolicy = agentMacro.get_actionMacro(macroState)
#         macroPolicy = agent.get_action(macroState)
        frame += 1
        tmpAction = []
        for i in range(0,rangeObsNumber):
            state = stateGenerator(rangeObsPos, [x,y], i)
            policyArr = agent.get_action(state)
            if i == 0:
                tmpAction = (1 - policyArr)
            else:
                tmpAction = tmpAction * (1 - policyArr)
#         print("0: ", tmpAction)
        
        if tmpAction != []:
            for j in range(0, action_size):
                if tmpAction[j] > 0.9999:
                    tmpAction[j] = 1
#                 elif tmpAction[j] > 0.995:
#                     tmpAction[j] = 0.1
                else:
                    tmpAction[j] = 0
            tmpArgMax = np.argmax(tmpAction)
            
#         tmpAction = [round(elem,0) for elem in tmpAction]
            
#         if rangeObsNumber != 0:
#             tmpAction = tmpAction * (1 - macroPolicy)
#         print("1:", macroPolicy)
    
        if rangeObsNumber == 0:
            tmpAction = [1.0/action_size for _ in range(0, action_size)]
        
        tmpGoalPos = goalFinder([x, y])
        state = stateGenerator(tmpGoalPos, [x,y], -1)
        policyArr = agent.get_action(state)

        if np.mean(tmpAction) == 0:
            tmpAction[tmpArgMax] = 1
        
        tmpAction = tmpAction * np.asarray(policyArr)
    
        tmpAction = tmpAction / np.sum(tmpAction)
        action = np.random.choice(action_size, 1, p = tmpAction)[0]

        xMove = 0
        yMove = 0

        [xMove, yMove] = takeAction(action)

        x = x + xMove
        y = y + yMove

        wallFlag = 0
        collisionFlag = 0
#         [x, y] = ckWall(x, y)
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)
#         pic = pygame.image.load("image/vacumrobot.jpg")
#         pic = pygame.transform.scale(pic, (40, 40))
#         screen.blit(pic, [x - agentRadius,y - agentRadius])
#         next_state = stateGenerator(obstaclePos, [x,y])\
        [rangeObsNumber, rangeObsPos, minIndex] = rangeFinder(obstaclePos, initPosAgent)
        next_macroState = stateGenerator([obstaclePos[minIndex][0], obstaclePos[minIndex][1]], [x, y], -1)

        initPosAgent = [x,y]
   
        if math.sqrt((x -  goalPos[0])**2 + (y - goalPos[1])**2) <= agentRadius:
            print("Goal Reached!")
            collisionFlag = 1
            done = True
        for i in range(0,obsNumber):
            if moveObstacles:
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,2)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,2)
                
            pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
            if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) < 19:
                print("Collision!")
                collisionFlag = -1
                print tmpAction
                done = True
#                 break
        
        if not done:
            reward = 0

        else:
            if collisionFlag == 1:
                reward = 0
                rList.append(1)
            elif collisionFlag == -1:
                reward = 1000
                rList.append(0)
#        if rangeObsNumber != 0:
#            agent.train_model(macroState, action, reward, next_macroState, done)
#        macroState = next_macroState
    
        score += reward
#         state = next_state
#         if score >= 10000:
#             print "Success!"
#             done = True
        pygame.draw.circle(screen, [255,100,100], initPosAgent, boundaryRadius, 2)

        if done:
            # every episode, plot the play time
            initPosAgent = initPosAgentStandard
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
            for i in range(0,obsNumber):
#                 obsRadius = random.randrange(agentRadius + obstacleRadius + 100, obstacleRandomRange)
#                 obsAngle = random.randrange(90,180) * math.pi / 180
#                 obstaclePos[i][0] = int(initPosAgent[0] + obsRadius * math.cos(obsAngle))
#                 obstaclePos[i][1] = int(initPosAgent[1] + obsRadius * math.sin(obsAngle))
#                 obstaclePos[i][0] = int(150 + i % 14 * 80)
#                 obstaclePos[i][1] = int(i / 14 * 80)
                obstaclePos[i][0] = int(150 + i % 7 * 150)
                obstaclePos[i][1] = int(150 + i / 7 * 150)
#                obstaclePos[i][0] = int(initPosAgent[0] - (obstacleRadius + agentRadius + random.randrange(0, dispSize[0])))
#                obstaclePos[i][1] = int(initPosAgent[1] + (obstacleRadius + agentRadius + random.randrange(0, dispSize[1])))

            episodes.append(e)
            
        #circle(Surface, color, pos, radius, width=0)
        pygame.draw.circle(screen, [100,255,100], goalPos, 10, 2)
        pygame.draw.circle(screen, [0, 255, 200], tmpGoalPos, 5, 5)
        pygame.display.flip()
        screen.fill([220,220,220])
        final = time.time()
        elapsed = final - start
        if frame >= 10:
            break
    print("Frame per second: " + str(frame / elapsed))
#    if e % 50 == 0:
#         agentMacro.actor.save_weights("./Practice004_DataSave/Actor_Macro.h5")
#         agentMacro.critic.save_weights("./Practice004_DataSave/Critic_Macro.h5")
#        agent.actor.save_weights("./Practice004_DataSave/Actor_Rev.h5")
#        agent.critic.save_weights("./Practice004_DataSave/Critic_Rev.h5")

    print score
    print("Percent of successful episodes: " + str(100.0 * sum(rList)/(e + 1)) + "%")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               384       
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 1161      
Total params: 1,545
Trainable params: 1,545
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               384       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 513
Trainable params: 513
Non-trainable params

/home/howoongjun/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


Frame per second: 187.645298247
0
Percent of successful episodes: 0.0%
('Episode ', 27, 'Starts!')
Frame per second: 189.200220131
0
Percent of successful episodes: 0.0%
('Episode ', 28, 'Starts!')
Frame per second: 224.713717044
0
Percent of successful episodes: 0.0%
('Episode ', 29, 'Starts!')
Frame per second: 204.128210869
0
Percent of successful episodes: 0.0%
('Episode ', 30, 'Starts!')
Frame per second: 222.201831946
0
Percent of successful episodes: 0.0%
('Episode ', 31, 'Starts!')
Frame per second: 208.272910098
0
Percent of successful episodes: 0.0%
('Episode ', 32, 'Starts!')
Frame per second: 213.01479924
0
Percent of successful episodes: 0.0%
('Episode ', 33, 'Starts!')
Frame per second: 222.618147849
0
Percent of successful episodes: 0.0%
('Episode ', 34, 'Starts!')
Frame per second: 219.972518539
0
Percent of successful episodes: 0.0%
('Episode ', 35, 'Starts!')
Frame per second: 217.456656989
0
Percent of successful episodes: 0.0%
('Episode ', 36, 'Starts!')
Frame per s

Frame per second: 201.033565475
0
Percent of successful episodes: 0.0%
('Episode ', 113, 'Starts!')
Goal Reached!
Frame per second: 172.524638714
0
Percent of successful episodes: 0.877192982456%
('Episode ', 114, 'Starts!')
Frame per second: 212.188254102
0
Percent of successful episodes: 0.869565217391%
('Episode ', 115, 'Starts!')
Frame per second: 220.803975658
0
Percent of successful episodes: 0.862068965517%
('Episode ', 116, 'Starts!')
Frame per second: 218.632110632
0
Percent of successful episodes: 0.854700854701%
('Episode ', 117, 'Starts!')
Frame per second: 227.303982138
0
Percent of successful episodes: 0.847457627119%
('Episode ', 118, 'Starts!')
Frame per second: 210.948192183
0
Percent of successful episodes: 0.840336134454%
('Episode ', 119, 'Starts!')
Frame per second: 231.948636557
0
Percent of successful episodes: 0.833333333333%
('Episode ', 120, 'Starts!')
Frame per second: 227.359428444
0
Percent of successful episodes: 0.826446280992%
('Episode ', 121, 'Starts!'

KeyboardInterrupt: 

In [9]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()

Percent of successful episodes: 95.2%
